In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install openpyxl==3.0.0    # upgrade from version 2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import random
import multiprocessing as mp
import dill
import progressbar
import datetime

%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install geopandas
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.7 MB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 54.7 MB/s 
     |████████████████████████████████| 1.0 MB 47.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.2 MB/s 


In [ ]:
root_path = '/content/drive/'
data_path = root_path + 'MyDrive/Distributionpaper/Enverus/'
result_path = root_path + 'MyDrive/Distributionpaper/WellData/'

# Enverus oil and gas production in 12 months

In [ ]:
def AssignMonthOilProd(i,row):
    if row['month'] == i:
        return row['Monthly Oil']

def AssignMonthGasProd(i,row):
    if row['month'] == i:
        return row['Monthly Gas']

In [ ]:
# Merge Enverus well headers and monthly production for a given year to create a data frame where oil and gas production from each month for each well is its own column
statename = 'CA_2017' # The filepath name for the relevant state in the relevant year, used for importing production data and well headers
# We vary the statename to produce the these data sheets for each region and year in question.
wellhead = pd.read_csv(data_path + statename + '_header Well Headers.CSV')
monprod = pd.read_csv(data_path + statename + ' Well Monthly Production.CSV')
monprod[["year", "month", "day"]] = monprod["Monthly Production Date"].str.split("-", expand = True)

merge = wellhead.merge(monprod[['API/UWI', 'Monthly Oil', 'Monthly Gas', 'month']], left_on = ['API14'], right_on = ['API/UWI'])
merge['Oil_01'] = 0
merge['Oil_02'] = 0
merge['Oil_03'] = 0
merge['Oil_04'] = 0
merge['Oil_05'] = 0
merge['Oil_06'] = 0
merge['Oil_07'] = 0
merge['Oil_08'] = 0
merge['Oil_09'] = 0
merge['Oil_10'] = 0
merge['Oil_11'] = 0
merge['Oil_12'] = 0
merge['Gas_01'] = 0
merge['Gas_02'] = 0
merge['Gas_03'] = 0
merge['Gas_04'] = 0
merge['Gas_05'] = 0
merge['Gas_06'] = 0
merge['Gas_07'] = 0
merge['Gas_08'] = 0
merge['Gas_09'] = 0
merge['Gas_10'] = 0
merge['Gas_11'] = 0
merge['Gas_12'] = 0

merge['Oil_01'] = merge.apply(lambda row: AssignMonthOilProd('01', row), axis=1)
merge['Oil_02'] = merge.apply(lambda row: AssignMonthOilProd('02', row), axis=1)
merge['Oil_03'] = merge.apply(lambda row: AssignMonthOilProd('03', row), axis=1)
merge['Oil_04'] = merge.apply(lambda row: AssignMonthOilProd('04', row), axis=1)
merge['Oil_05'] = merge.apply(lambda row: AssignMonthOilProd('05', row), axis=1)
merge['Oil_06'] = merge.apply(lambda row: AssignMonthOilProd('06', row), axis=1)
merge['Oil_07'] = merge.apply(lambda row: AssignMonthOilProd('07', row), axis=1)
merge['Oil_08'] = merge.apply(lambda row: AssignMonthOilProd('08', row), axis=1)
merge['Oil_09'] = merge.apply(lambda row: AssignMonthOilProd('09', row), axis=1)
merge['Oil_10'] = merge.apply(lambda row: AssignMonthOilProd('10', row), axis=1)
merge['Oil_11'] = merge.apply(lambda row: AssignMonthOilProd('11', row), axis=1)
merge['Oil_12'] = merge.apply(lambda row: AssignMonthOilProd('12', row), axis=1)
merge['Gas_01'] = merge.apply(lambda row: AssignMonthGasProd('01', row), axis=1)
merge['Gas_02'] = merge.apply(lambda row: AssignMonthGasProd('02', row), axis=1)
merge['Gas_03'] = merge.apply(lambda row: AssignMonthGasProd('03', row), axis=1)
merge['Gas_04'] = merge.apply(lambda row: AssignMonthGasProd('04', row), axis=1)
merge['Gas_05'] = merge.apply(lambda row: AssignMonthGasProd('05', row), axis=1)
merge['Gas_06'] = merge.apply(lambda row: AssignMonthGasProd('06', row), axis=1)
merge['Gas_07'] = merge.apply(lambda row: AssignMonthGasProd('07', row), axis=1)
merge['Gas_08'] = merge.apply(lambda row: AssignMonthGasProd('08', row), axis=1)
merge['Gas_09'] = merge.apply(lambda row: AssignMonthGasProd('09', row), axis=1)
merge['Gas_10'] = merge.apply(lambda row: AssignMonthGasProd('10', row), axis=1)
merge['Gas_11'] = merge.apply(lambda row: AssignMonthGasProd('11', row), axis=1)
merge['Gas_12'] = merge.apply(lambda row: AssignMonthGasProd('12', row), axis=1)


merge1 = merge[['API14', 'Surface Hole Latitude (WGS84)', 'Surface Hole Longitude (WGS84)', 'Oil_01', 'Oil_02', 'Oil_03', 'Oil_04', 'Oil_05', 'Oil_06', 'Oil_07', 'Oil_08', 'Oil_09', 'Oil_10', 'Oil_11', 'Oil_12', 'Gas_01', 'Gas_02', 'Gas_03', 'Gas_04', 'Gas_05', 'Gas_06', 'Gas_07', 'Gas_08', 'Gas_09', 'Gas_10', 'Gas_11', 'Gas_12']]
newdf = merge1.groupby('API14').agg({'Surface Hole Latitude (WGS84)': 'first', 'Surface Hole Longitude (WGS84)': 'first', 'Oil_01': np.sum, 'Oil_02': np.sum, 'Oil_03': np.sum, 'Oil_04': np.sum, 'Oil_05': np.sum, 'Oil_06': np.sum, 'Oil_07': np.sum, 'Oil_08': np.sum, 'Oil_09': np.sum, 'Oil_10': np.sum, 'Oil_11': np.sum, 'Oil_12': np.sum, 'Gas_01': np.sum, 'Gas_02': np.sum, 'Gas_03': np.sum, 'Gas_04': np.sum, 'Gas_05': np.sum, 'Gas_06': np.sum, 'Gas_07': np.sum, 'Gas_08': np.sum, 'Gas_09': np.sum, 'Gas_10': np.sum, 'Gas_11': np.sum, 'Gas_12': np.sum})
newdf = newdf.reset_index()
newdf = newdf.sort_values(by=['API14'])
newdf.to_csv(result_path + statename + '_welldata.csv', index=False)
newdf

,API14,Surface Hole Latitude (WGS84),Surface Hole Longitude (WGS84),Oil_01,Oil_02,Oil_03,Oil_04,Oil_05,Oil_06,Oil_07,...,Gas_03,Gas_04,Gas_05,Gas_06,Gas_07,Gas_08,Gas_09,Gas_10,Gas_11,Gas_12
0,4001000010000,37.693672,-121.689629,9.0,9.0,3.0,2.0,0.0,20.0,0.0,...,0.0,0.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
1,4001200040000,37.693733,-121.689697,95.0,89.0,113.0,20.0,0.0,39.0,0.0,...,7.0,4.0,6.0,7.0,0.0,0.0,0.0,8.0,7.0,7.0
2,4001200090000,37.694962,-121.686813,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4001200120000,37.693443,-121.684006,272.0,275.0,346.0,386.0,360.0,221.0,430.0,...,28.0,27.0,23.0,23.0,35.0,32.0,21.0,21.0,17.0,19.0
4,4001200410000,37.692451,-121.684547,66.0,62.0,19.0,94.0,90.0,51.0,0.0,...,2.0,6.0,6.0,5.0,0.0,0.0,0.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51349,4259217460000,33.662441,-118.044395,976.0,2213.0,2447.0,2687.0,2986.0,2883.0,2972.0,...,913.0,933.0,1330.0,1268.0,1257.0,1323.0,1185.0,1130.0,1126.0,1078.0
51350,4259217540000,33.662483,-118.044571,1395.0,3962.0,4167.0,3992.0,4399.0,4171.0,4510.0,...,1555.0,1386.0,1958.0,1835.0,1908.0,2033.0,1893.0,2036.0,1899.0,1869.0
51351,4259217550000,33.662483,-118.044556,4543.0,3609.0,3622.0,3538.0,3719.0,3371.0,2299.0,...,1352.0,1228.0,1656.0,1483.0,973.0,55.0,61.0,102.0,78.0,0.0
51352,4259217560000,33.662468,-118.044571,4244.0,3445.0,3487.0,3399.0,3651.0,3214.0,3515.0,...,1302.0,1180.0,1625.0,1414.0,1487.0,1722.0,1630.0,1756.0,1558.0,1497.0


In [ ]:
# The same basic thing for Texas data in 2019

statename = 'TX_2019_V'
wellhead = pd.read_csv(data_path + statename + '_header Well Headers.CSV')
monprod = pd.read_csv(data_path + statename + ' Well Monthly Production.CSV')
monprod[["year", "month", "day"]] = monprod["Monthly Production Date"].str.split("-", expand = True)

merge = wellhead.merge(monprod[['API/UWI', 'Monthly Oil', 'Monthly Gas', 'month']], left_on = ['API14'], right_on = ['API/UWI'])

In [ ]:
merge = merge[['API14', 'Surface Hole Latitude (WGS84)', 'Surface Hole Longitude (WGS84)', 'Monthly Oil', 'Monthly Gas', 'month']]
merge['Oil_01'] = 0
merge['Oil_02'] = 0
merge['Oil_03'] = 0
merge['Oil_04'] = 0
merge['Oil_05'] = 0
merge['Oil_06'] = 0
merge['Oil_07'] = 0
merge['Oil_08'] = 0
merge['Oil_09'] = 0
merge['Oil_10'] = 0
merge['Oil_11'] = 0
merge['Oil_12'] = 0
merge['Gas_01'] = 0
merge['Gas_02'] = 0
merge['Gas_03'] = 0
merge['Gas_04'] = 0
merge['Gas_05'] = 0
merge['Gas_06'] = 0
merge['Gas_07'] = 0
merge['Gas_08'] = 0
merge['Gas_09'] = 0
merge['Gas_10'] = 0
merge['Gas_11'] = 0
merge['Gas_12'] = 0

merge['Oil_01'] = merge.apply(lambda row: AssignMonthOilProd('01', row), axis=1)
merge['Oil_02'] = merge.apply(lambda row: AssignMonthOilProd('02', row), axis=1)
merge['Oil_03'] = merge.apply(lambda row: AssignMonthOilProd('03', row), axis=1)
merge['Oil_04'] = merge.apply(lambda row: AssignMonthOilProd('04', row), axis=1)
merge['Oil_05'] = merge.apply(lambda row: AssignMonthOilProd('05', row), axis=1)
merge['Oil_06'] = merge.apply(lambda row: AssignMonthOilProd('06', row), axis=1)
merge['Oil_07'] = merge.apply(lambda row: AssignMonthOilProd('07', row), axis=1)
merge['Oil_08'] = merge.apply(lambda row: AssignMonthOilProd('08', row), axis=1)
merge['Oil_09'] = merge.apply(lambda row: AssignMonthOilProd('09', row), axis=1)
merge['Oil_10'] = merge.apply(lambda row: AssignMonthOilProd('10', row), axis=1)
merge['Oil_11'] = merge.apply(lambda row: AssignMonthOilProd('11', row), axis=1)
merge['Oil_12'] = merge.apply(lambda row: AssignMonthOilProd('12', row), axis=1)
merge['Gas_01'] = merge.apply(lambda row: AssignMonthGasProd('01', row), axis=1)
merge['Gas_02'] = merge.apply(lambda row: AssignMonthGasProd('02', row), axis=1)
merge['Gas_03'] = merge.apply(lambda row: AssignMonthGasProd('03', row), axis=1)
merge['Gas_04'] = merge.apply(lambda row: AssignMonthGasProd('04', row), axis=1)
merge['Gas_05'] = merge.apply(lambda row: AssignMonthGasProd('05', row), axis=1)
merge['Gas_06'] = merge.apply(lambda row: AssignMonthGasProd('06', row), axis=1)
merge['Gas_07'] = merge.apply(lambda row: AssignMonthGasProd('07', row), axis=1)
merge['Gas_08'] = merge.apply(lambda row: AssignMonthGasProd('08', row), axis=1)
merge['Gas_09'] = merge.apply(lambda row: AssignMonthGasProd('09', row), axis=1)
merge['Gas_10'] = merge.apply(lambda row: AssignMonthGasProd('10', row), axis=1)
merge['Gas_11'] = merge.apply(lambda row: AssignMonthGasProd('11', row), axis=1)
merge['Gas_12'] = merge.apply(lambda row: AssignMonthGasProd('12', row), axis=1)
merge1 = merge[['API14', 'Surface Hole Latitude (WGS84)', 'Surface Hole Longitude (WGS84)', 'Oil_01', 'Oil_02', 'Oil_03', 'Oil_04', 'Oil_05', 'Oil_06', 'Oil_07', 'Oil_08', 'Oil_09', 'Oil_10', 'Oil_11', 'Oil_12', 'Gas_01', 'Gas_02', 'Gas_03', 'Gas_04', 'Gas_05', 'Gas_06', 'Gas_07', 'Gas_08', 'Gas_09', 'Gas_10', 'Gas_11', 'Gas_12']]
newdf = merge1.groupby('API14').agg({'Surface Hole Latitude (WGS84)': 'first', 'Surface Hole Longitude (WGS84)': 'first', 'Oil_01': np.sum, 'Oil_02': np.sum, 'Oil_03': np.sum, 'Oil_04': np.sum, 'Oil_05': np.sum, 'Oil_06': np.sum, 'Oil_07': np.sum, 'Oil_08': np.sum, 'Oil_09': np.sum, 'Oil_10': np.sum, 'Oil_11': np.sum, 'Oil_12': np.sum, 'Gas_01': np.sum, 'Gas_02': np.sum, 'Gas_03': np.sum, 'Gas_04': np.sum, 'Gas_05': np.sum, 'Gas_06': np.sum, 'Gas_07': np.sum, 'Gas_08': np.sum, 'Gas_09': np.sum, 'Gas_10': np.sum, 'Gas_11': np.sum, 'Gas_12': np.sum})
newdf = newdf.reset_index()
newdf = newdf.sort_values(by=['API14'])
newdf.to_csv(result_path + statename + '_welldata.csv', index=False)
newdf

,API14,Surface Hole Latitude (WGS84),Surface Hole Longitude (WGS84),Oil_01,Oil_02,Oil_03,Oil_04,Oil_05,Oil_06,Oil_07,...,Gas_03,Gas_04,Gas_05,Gas_06,Gas_07,Gas_08,Gas_09,Gas_10,Gas_11,Gas_12
0,42001000460000,32.011518,-95.983356,47.0,42.0,39.0,0.0,56.0,36.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,42001000710000,32.052814,-95.645865,17.0,15.0,15.0,13.0,14.0,12.0,12.0,...,10033.0,11934.0,11757.0,9458.0,9301.0,8748.0,10107.0,11774.0,10609.0,10867.0
2,42001000900000,32.062775,-95.563729,92.0,52.0,14.0,30.0,36.0,14.0,173.0,...,1031.0,1418.0,1982.0,1765.0,1283.0,1067.0,993.0,989.0,1222.0,1790.0
3,42001001150000,32.043215,-95.539422,16.0,14.0,14.0,13.0,14.0,12.0,12.0,...,241.0,288.0,284.0,230.0,226.0,214.0,247.0,289.0,261.0,269.0
4,42001002730000,31.920936,-95.426127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,10.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178120,42602302080000,27.680537,-97.088215,1469.0,1234.0,1382.0,1330.0,1280.0,1153.0,1183.0,...,49111.0,49194.0,50306.0,44729.0,47463.0,35514.0,46857.0,49508.0,48274.0,48978.0
178121,42702302300000,27.757093,-96.982268,510.0,517.0,510.0,467.0,472.0,472.0,453.0,...,41660.0,39758.0,38244.0,39544.0,38165.0,26258.0,37843.0,39865.0,37607.0,37414.0
178122,42703302170000,28.055756,-96.721720,183.0,168.0,146.0,182.0,182.0,86.0,113.0,...,1106.0,1745.0,1813.0,908.0,642.0,1933.0,1741.0,1869.0,1272.0,1592.0
178123,42708303760000,29.515895,-94.145643,0.0,2694.0,0.0,0.0,0.0,0.0,0.0,...,57068.0,90923.0,44690.0,41178.0,66092.0,64499.0,48142.0,48310.0,35716.0,57880.0


# Combine Permian well data

In [ ]:
# Combines Texas and New Mexico data for 2020 campaigns spanning the full permian
df1 = pd.read_csv(result_path + 'NM_2020' + '_welldata.csv')
df2 = pd.read_csv(result_path + 'TX_2020_V' + '_welldata.csv')
df3 = pd.read_csv(result_path + 'TX_2020_DUH' + '_welldata.csv')
print('total rows: ' + str(len(df1)+len(df2)+len(df3)))

total rows: 342674


In [ ]:
resultdf = pd.concat([df1, df2, df3])
print(len(resultdf))
# resultdf.to_csv(result_path + 'Permian_2021' + '_welldata.csv', index=False)
resultdf

342674


,API14,Surface Hole Latitude (WGS84),Surface Hole Longitude (WGS84),Oil_01,Oil_02,Oil_03,Oil_04,Oil_05,Oil_06,Oil_07,...,Gas_03,Gas_04,Gas_05,Gas_06,Gas_07,Gas_08,Gas_09,Gas_10,Gas_11,Gas_12
0,30005000420000,33.443131,-104.416794,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30005000440000,33.439526,-104.416283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30005000520000,33.425697,-104.387894,39.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30005000620000,33.411919,-104.395142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30005000670000,33.418430,-104.392029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75880,42703302780000,28.055758,-96.721751,1968.0,1228.0,2041.0,1931.0,2198.0,2359.0,1712.0,...,682.0,644.0,566.0,578.0,455.0,352.0,497.0,559.0,359.0,500.0
75881,42703302800000,28.055724,-96.721747,2301.0,1121.0,1813.0,1792.0,1837.0,1446.0,990.0,...,149.0,0.0,946.0,835.0,595.0,487.0,707.0,794.0,580.0,855.0
75882,42706302360000,29.126466,-94.821870,3373.0,3546.0,3323.0,3279.0,0.0,1058.0,3578.0,...,6786.0,4167.0,0.0,2180.0,7311.0,6193.0,6052.0,6621.0,6584.0,5555.0
75883,42706302500100,29.145806,-94.830732,6471.0,5931.0,6210.0,5406.0,0.0,3169.0,6254.0,...,3636.0,4280.0,0.0,1275.0,3352.0,3041.0,2797.0,3045.0,2458.0,2573.0
